In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from pathlib import Path
from time import sleep

DRIVER = Path('./geckodriver').absolute()

In [2]:
URL = 'https://www.iucnredlist.org/search/list'

In [3]:
# access one country with selenium
driver = webdriver.Firefox(executable_path=DRIVER)
driver.get(URL)
search_box = driver.find_element_by_class_name('search--site')
search_button = driver.find_element_by_class_name('search--site__button__key')
search_box.send_keys('Austria')
search_button.click()
# wait for search
sleep(10)
main_content = driver.find_element_by_class_name('layout-page__major')
main_html = main_content.get_attribute('innerHTML')

In [4]:
# get html with beautifulsoup
soup = BeautifulSoup(main_html, 'html.parser')
species = soup.find_all('li', class_='list-results__item')
species

[<li class="list-results__item"><span class="list-results__classification">plantae — magnoliopsida</span><h2 class="list-results__title">Austrian Field Cress</h2><p class="list-results__subtitle"><em>Rorippa austriaca</em></p><span class="species-population -sm species-population--unknown">Unknown</span><span class="species-assessment u--capitalize">Global</span><a class="species-category -sm -icon species-category--lc" href="/species/176580/42339673" title="Least Concern">lc</a><a aria-hidden="true" class="link--faux" href="/species/176580/42339673" tabindex="-1">View</a></li>,
 <li class="list-results__item"><span class="list-results__classification">plantae — liliopsida</span><h2 class="list-results__title">Austrian Gymnadenia </h2><p class="list-results__subtitle"><em>Gymnadenia archiducis-joannis</em></p><span class="species-population -sm species-population--decreasing">Decreasing</span><span class="species-assessment u--capitalize">Global, Europe</span><a class="species-category

In [5]:
len(species)
# expected 18 results

18

In [6]:
# next task extrakt the information by species